In [ ]:
!pip install soynlp

In [ ]:
!pip show soynlp

In [ ]:
import pandas as pd
import numpy as np
import re# 정규표현식

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/국민청원분석/petition.csv', index_col=0, parse_dates=['start', 'end'])

In [ ]:
data.shape

In [ ]:
data.tail(10)

In [ ]:
# 내 관심사 키워드를 통애 데이터 가져오기
p = r'.*(스포츠|야구|대학|취업).*'
care = data[data['title'].str.match(p) |
           data['content'].str.match(p, flags=re.MULTILINE)]
care.shape

In [ ]:
care.tail(50)

In [ ]:
# 491514번을 자세히 보면

In [ ]:
test_idx = 491514

In [ ]:
sample_title = care.loc[test_idx, 'title']
sample_title

In [ ]:
sample_content = care['content'][test_idx]
sample_content

In [ ]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()
tokenizer

In [ ]:
# 단어 분리
tokened_title = tokenizer.tokenize(sample_title)
tokened_title

In [ ]:
# 내용 단어 분리
tokened_content = tokenizer.tokenize(sample_content)
tokened_content[:20]

In [ ]:
print(len(tokened_title))
print(len(tokened_content))

In [ ]:
# 데이터 전처리

def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    return text

In [ ]:
%time sentences = care['content'].apply(preprocessing)
# %time을 통해 코드 실행 시간 출력

In [ ]:
%time tokens = sentences.apply(tokenizer.tokenize)
tokens

In [ ]:
tokens[test_idx][:10]

In [ ]:
# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

In [ ]:
# matplotlib 한글 폰트 관련

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

%matplotlib inline

mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
print('버전 : ', mpl.__version__)
print('설치 위치 : ', mpl.__file__)
print('설정 위치 : ', mpl.get_configdir())
print('캐시 위치 : ', mpl.get_cachedir())

In [ ]:
[ (f.name, f.fname) for f in fm.fontManager.ttflist if 'NaNum' in f.name]

In [ ]:
path = '/content/drive/MyDrive/Korean_Fonts/Nanum/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname = path, size = 18)# 이후 도식화할때 fontproperties = fontprop

In [ ]:
stopwords_kr = ['하지만', '그리고', '그런데', '저는','제가','그럼', '이런', '저런', '합니다','많은', '많이', '정말', '너무'] 

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

def display_Wordcloud(data = None, backgroundcolor = 'white', width = 800, height = 600):
    wordcloud = WordCloud( font_path = path, stopwords = stopwords_kr, background_color = backgroundcolor, width = width, height = height).generate(data)

    plt.figure(figsize = (15, 10))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [ ]:
%time display_Wordcloud(' '.join(sentences))

In [ ]:
# 사실상 불용어가 많이 등장해서, 워드클라우드의 이미를 찾기 힘드므로, 이를 해결하기 위해 명사추출
from soynlp.noun import LRNounExtractor

In [ ]:
%%time
noun_extractor = LRNounExtractor(verbose = True)
noun_extractor.train(sentences)
nouns = noun_extractor.extract()

In [ ]:
%time display_Wordcloud(' '.join(nouns))

## WordCloud를 만들긴 하였으나, 불용어가 많아서 의미를 찾기가 어려우니 wordtovec으로 단어 유사도를 찾거나, 불용어내용을 늘리거나 하는 다른 방법으로 워드클라우드를 개선해야할 듯